<a href="https://www.kaggle.com/code/dayyak/student-performance-multiple-linear-regression?scriptVersionId=188815228" target="_blank"><img align="left" alt="Kaggle" title="Open in Kaggle" src="https://kaggle.com/static/images/open-in-kaggle.svg"></a>

# Introduction

This analysis aims to explore the relationship between students' lifestyles, study techniques, and their performance scores. Based on this relationship, I will develop a model designed to predict student performance and achieve the most accurate results possible.

# Data Analysis and Processing

In [ ]:
# Import Python libraries
import warnings
import numpy as np 
import pandas as pd 
import seaborn as sns
import scipy.stats as sts
import matplotlib.pyplot as plt

from pandas.plotting import scatter_matrix

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_squared_error, r2_score

warnings.filterwarnings("ignore")

In [ ]:
# Load data
df = pd.read_csv('/kaggle/input/student-performance-multiple-linear-regression/Student_Performance.csv')
df.head()

In [ ]:
df.info()

In [ ]:
df.describe()

In [ ]:
# visualizing the features
df.hist(bins=20, figsize=(20, 15))
plt.show()

After exploring the dataset and examining the distribution of features, I found that the data is almost balanced. Most features are distributed symmetrically with mean values near the center, indicating that the data is well-balanced. The dataset is clean with no missing values, and the features do not require transformation or cleaning. All independent features, except for one, are numeric integers. Based on this analysis, the only necessary preprocessing step is to convert the single categorical column into an integer or boolean format.

**Convert the (Extracurricular Activities) feature into a dummy variable.**

In [ ]:
df['Extracurricular Activities'] = df['Extracurricular Activities'].map({'Yes': 1, 'No': 0})

In [ ]:
activity_counts = df['Extracurricular Activities'].value_counts()
labels = ['No', 'Yes']
colors = ['lightcoral', 'lightskyblue']
explode = (0.1, 0) 

plt.pie(activity_counts, labels=labels, colors=colors, autopct='%1.1f%%', startangle=140, explode=explode)
plt.title('Proportion of Students Participating in Extracurricular Activities')
plt.axis('equal')

plt.show()

**Let's determine the correlation between the columns.**

Before calculating correlations, let's visualize the relationship between each feature and the performance feature by creating a box plot for each feature.

In [ ]:
fig, axes = plt.subplots(nrows=2, ncols=2, figsize=(12, 8))  # 2x2 grid of subplots
fig.suptitle('Comparison of Independent Features with Dependent Feature (Box Plots)', fontsize=16)

# Plotting each subplot
sns.boxplot(x='Hours Studied', y='Performance Index', data=df, ax=axes[0, 0])
axes[0, 0].set_title('Hours Studied')

sns.boxplot(x='Previous Scores', y='Performance Index', data=df, ax=axes[0, 1])
axes[0, 1].set_title('Previous Scores')

sns.boxplot(x='Sleep Hours', y='Performance Index', data=df, ax=axes[1, 0])
axes[1, 0].set_title('Sleep Hours')

sns.boxplot(x='Sample Question Papers Practiced', y='Performance Index', data=df, ax=axes[1, 1])
axes[1, 1].set_title('Sample Question Papers Practiced')

plt.tight_layout()
plt.show()

A slight increase in performance is observed in the plot of "Hours Studied", and a more significant increase is seen in the second plot for "Previous Scores". However, no significant effect is apparent in the bottom plots for the features "Sleep Hours" and "Sample Question Papers Practiced". 

*Let's evaluate this using correlation analysis.*

In [ ]:
corr = df.corr()

# Generate a heatmap
plt.figure(figsize=(8, 6))
sns.heatmap(corr, annot=True, cmap='coolwarm', fmt='.2f', linewidths=0.5)
plt.title('Feature Correlation Heatmap')
plt.show()

A strong positive correlation is observed between student performance and previous scores, and a moderate positive correlation is seen with studying hours. However, no significant relationship is found with other features such as extracurricular activities, sleeping hours, and the number of sample question papers practiced.

# Construct and validate the model

In [ ]:
X = df.drop(columns='Performance Index')
y = df['Performance Index']

reg_model = LinearRegression()

cv_scores = cross_val_score(reg_model, X, y, cv=10, scoring='neg_mean_squared_error')

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

reg_model.fit(X_train, y_train)

I split the data into 80% for training and 20% for testing. Prior to this, I used cross-validation with 10 folds to evaluate the model and took the mean of the results.

# Result

In [ ]:
# Evaluate the final model on the test set
print(f'Average MSE from 10-fold cross-validation: {-cv_scores.mean():.2f}')
print(f'Test set R^2 score: {reg_model.score(X_test, y_test):.2f}')

Based on the results obtained, the model demonstrates strong performance across evaluations. The average Mean Squared Error (MSE) from 10-fold cross-validation is 4.15, indicating that the average difference between the actual and predicted values is only 4.15 marks, which is good. Additionally, the model achieves an impressive R² score of 0.99 on the test set, highlighting its capability to accurately explain the variance in the target variable.

*Let's visualize the predicted and actual values of y to see how they align.*

In [ ]:
y_predict = reg_model.predict(X_test)

# Assuming y_pred and y_test are your prediction and test target data
sns.scatterplot(x=y_predict, y=y_test, color='red', label='Actual vs. Predicted')

# Adding labels and title
plt.xlabel('Predicted Values')
plt.ylabel('True Values')
plt.title('Scatter Plot of Actual vs. Predicted Values')
plt.legend()
plt.grid(True)
plt.show()

As we can see, the plot of predicted and actual values forms an almost straight 45-degree line, indicating that the model's predictions are highly accurate. This close match between predicted and actual values suggests that the model is performing very well with a high level of accuracy.

*Now, I will analyze the residuals to determine if they meet the criteria for a normal distribution.*

In [ ]:
residuals = y_test - y_predict
print(residuals)

In [ ]:
sns.displot(residuals, kde=True)

In [ ]:
plt.figure(figsize=(10, 6))
sts.probplot(residuals, dist="norm", plot=plt)
plt.title('Q-Q Plot of Residuals')
plt.grid(True)
plt.show()

In [ ]:
# Perform the Shapiro-Wilk test
stat, p_value = sts.shapiro(residuals)
print(f'Shapiro-Wilk Test statistic: {stat:.2f}')
print(f'p-value: {p_value:.2f}')

As we see, residuals are normaly distributed which means that the model captures the linear relationship between the independent featurs and the target variable accurately. The estimates of the model coefficients are unbiased. 

# Summary

In this analysis, the dependent variable "Performance" was examined. The residuals were found to follow a normal distribution, supported by a high p-value of 0.98. The model's performance was evaluated using a 10-fold cross-validation, yielding an average Mean Squared Error (MSE) of 4.15, indicating consistent accuracy across various data subsets. Moreover, the model achieved an impressive R² score of 0.99 on the test set, demonstrating its robust capability to explain the variability in "Performance Index". Overall, these findings affirm that the model performs exceptionally well in predicting the dependent variable with high accuracy and reliability.